<a href="https://colab.research.google.com/github/zoeschmilovich/workshop_intro-to-gwas-and-prs/blob/main/Part_2_Fundamentals_of_polygenic_risk_scores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computing individual-level polygenic risk scores

# Set up command-line environment

In [ ]:
!git clone https://github.com/zoeschmilovich/workshop_intro-to-gwas-and-prs.git

Download GWAS summary statistics

In [ ]:
!mkdir /content/workshop_intro-to-gwas-and-prs/data/gwas-sum-stats

!wget https://ftp.ebi.ac.uk/pub/databases/gwas/summary_statistics/GCST007001-GCST008000/GCST007245/CousminerDL_30254083.gz /content/workshop_intro-to-gwas-and-prs/data/gwas-sum-stats/T2D_gwas

In [ ]:
%%capture

# Download PLINK2
!wget -qO- https://s3.amazonaws.com/plink2-assets/plink2_linux_x86_64_latest.zip > plink2.zip
!unzip -q plink2.zip
!mv plink2 /usr/local/bin/
!plink2 --version

In [ ]:
# Define the directory
dir="/content/workshop_intro-to-gwas-and-prs"

In [ ]:
# Add mock binary (affected (2) vs. unaffected (1)) phenotypes to the PLINK files

!plink2 --bfile $dir/data/genotypes/EUR \
        --pheno $dir/data/genotypes/T2D_phenotypes.txt \
        --make-bed \
        --out $dir/data/genotypes/T2D_case_control

# Count the number of
!plink2 --bfile $dir/data/genotypes/T2D_case_control \
        --freq \
        --out $dir/data/genotypes/case_control_counts

The `afreq` file contains allele frequency information for the variants across our samples.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the .afreq file into a pandas DataFrame
afreq_df = pd.read_csv(f"{dir}/data/genotypes/case_control_counts.afreq", delim_whitespace=True)

# Display the first few rows of the DataFrame
print(afreq_df.head())

plt.figure(figsize=(10, 6))
plt.hist(afreq_df['ALT_FREQS'], bins=50, edgecolor='k', alpha=0.7)
plt.axvline(x=0.01, color='red', linestyle='--', linewidth=1.5, label='MAF 0.01')

plt.xlabel('Alternative Allele Frequency')
plt.ylabel('Number of Variants')
plt.title('Distribution of Alternative Allele Frequencies')
plt.grid(True)
plt.legend()
plt.show()


Explanation of filtering:
--maf 0.01 \ #  Filters SNPs to include only those with a minor allele frequency (MAF) above 0.01.
--geno 0.1 \ # Excludes SNPs with missing call rates exceeding 0.1.
--hwe 0.0001 \ # Excludes SNPs with Hardy-Weinberg equilibrium p-values below 0.0001.
--make-bed \ # Output will be in PLINK (.bed, .bim, .fam) format
--out $dir/EUR_maf-0.01_geno-0.1_hwe-1e4 # QC'ed output PLINK files


In [ ]:
# Perform initial filtering by MAF, SNP call rate, exclude specific SNPs, and keep specific samples

!plink2 --bfile $dir/data/genotypes/EUR \
--maf 0.01 \
--geno 0.1 \
--hwe 0.0001 \
--make-bed \
--out $dir/data/genotypes/EUR_maf-0.01_geno-0.1_hwe-1e4

# Perform LD pruning in two steps:
# Step 1: Generate list of SNPs to keep
# We perform linkage disequilibrium (LD) pruning using:
  # - a window size of 50 SNPs
  # - a step size of 5 SNPs
  # - r2 threshold of 0.2 (correlation between SNPs)
!plink2 --bfile $dir/data/genotypes/EUR_maf-0.01_geno-0.1_hwe-1e4 \
       --indep-pairwise 50 5 0.2 \
       --out $dir/data/genotypes/EUR_ld_pruned

# Step 2: Apply LD pruning
!plink2 --bfile $dir/data/genotypes/EUR_maf-0.01_geno-0.1_hwe-1e4 \
       --extract $dir/data/genotypes/EUR_ld_pruned.prune.in \
       --make-bed \
       --out $dir/data/genotypes/EUR_QCed

Install dependencies needed for PRS-Cs to compute PRS

In [ ]:
# Clone the PRS-CS repository
!git clone https://github.com/getian107/PRScs.git

# Navigate to the PRS-CS directory
%cd PRScs

# Download the pre-trained LD reference panels
!wget https://personal.broadinstitute.org/hhuang//public/PRS-CSx/Reference/1KG/ldblk_1kg_eur.tar.gz

# Extract the downloaded files
!tar -xzvf ldblk_1kg_eur.tar.gz

Run PRScs to estimate the effect size of SNPs

In [ ]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Number of SNPs to simulate
num_snps = 10000

# Generate mock data
data = {
    'SNP': [f'rs{1000000 + i}' for i in range(num_snps)],  # SNP IDs
    'CHR': np.random.randint(1, 23, size=num_snps),        # Chromosome number (1-22)
    'BP': np.random.randint(1, 1e6, size=num_snps),        # Base pair position
    'A1': np.random.choice(['A', 'C', 'G', 'T'], size=num_snps),  # Effect allele
    'A2': np.random.choice(['A', 'C', 'G', 'T'], size=num_snps),  # Non-effect allele
    'Freq1': np.random.uniform(0.01, 0.5, size=num_snps),  # Allele frequency of A1
    'b': np.random.normal(0, 0.1, size=num_snps),          # Estimated effect size (beta)
    'se': np.random.uniform(0.01, 0.1, size=num_snps),     # Standard error of beta
    'p': np.random.uniform(0, 1, size=num_snps)            # P-value
}

# Create DataFrame
gwas_df = pd.DataFrame(data)

# Adjust the p-values to make some SNPs significant
significant_snps = np.random.choice(num_snps, size=int(num_snps * 0.05), replace=False)
gwas_df.loc[significant_snps, 'p'] = gwas_df.loc[significant_snps, 'p'] / 1e6

# Save to a file
gwas_df.to_csv(f"{dir}/data/gwas_summary_stats.txt', sep='\t', index=False)")


In [ ]:
# Example command to run PRS-CS

!mkdir $dir/output/PRScs-estimates

!python PRScs.py \
        --ref_dir=/content/PRScs/ldblk_1kg_eur \
        --bim_prefix=$dir/data/genotypes/EUR_QCed \
        --sst_file=$dir/data/gwas_summary_stats.txt \
        --n_gwas=100000 \
        --out_dir=$dir/output/PRScs-estimates/EUR

*Following PRScs, *

In [ ]:
# Concatenate all chromosome files for LBC

!cat $dir/output/PRScs-estimates/EUR_pst_eff_a1_b0.5_phiauto_chr*.txt | sort -n -k1 > $dir/output/PRScs-estimates/EUR_pst_eff_a1_b0.5_phiauto_allchr.txt

# Prepare the output of PRScs so that it can be used by PLINK
!awk '{print $2, $4, $6}' \
      /content/workshop_intro-to-gwas-and-prs/output/PRScs-estimates/EUR_pst_eff_a1_b0.5_phiauto_allchr.txt | \
      sed 's/ /\t/g' > \
      /content/workshop_intro-to-gwas-and-prs/output/PRScs-estimates/EUR_pst_eff_a1_b0.5_phiauto_allchr.PLINKscore


In [ ]:
# Run PLINK2 to calculate PRS


!plink2 \
    --bfile $dir/data/genotypes/EUR_QCed \
    --pheno $dir/data/genotypes/T2D_phenotypes.txt \
    --missing \
    --out $dir/data/case_control_counts


# Generate a frequency report including phenotype information
!plink2 --bfile $dir/data/genotypes/EUR_QCed \
       --pheno $dir/data/genotypes/T2D_phenotypes.txt \
       --freq case-control \
       --out $dir/data/case_control_counts

!less $dir/data/case_control_counts.frq.cc

!less /content/workshop_intro-to-gwas-and-prs/data/case_control_counts.smiss

!plink2 \
    --bfile $dir/data/genotypes/EUR_QCed \
    --score $dir/output_pst_eff_a1_b0.5_phiauto_allchr.PLINKscore no-mean-imputation \
    --out $dir/PRS_scores